## Course: TM10007 - Machine learning
Editors: Lishia Vergeer, Amy Roos, Maaike Pruijt, Hilde Roording.

Description: The aim of this code is to predict the tumor grade of glioma’s(high or low) before surgery, 
based on features extracted from a combination of four MRI images: 
T2-weighted, T2-weighted FLAIR and T1-weighted before and after injection of contrast agent.

#### Import packages

In [1]:
# General packages
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets as ds

# Import code
from brats.load_data import load_data

# Performance 
from sklearn.model_selection import train_test_split
from sklearn import decomposition
import seaborn


# Pipeline and gridsearch
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

#preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
# scaler
from sklearn.preprocessing import RobustScaler

#Machine learning classifiers
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn import feature_selection 
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.model_selection import ShuffleSplit
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neighbors import KNeighborsRegressor



C:\Users\hilde\miniconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


#### Load data

In [3]:
data_brats = load_data()

# Convert to dataframe
X = pd.DataFrame(data_brats)

# Print data 
print(f'The number of samples in data_brats: {len(X.index)}')
print(f'The number of columns in data_brats: {len(X.columns)}')

The number of samples in data_brats: 167
The number of columns in data_brats: 725


c:\Users\hilde\Python\TM10007_Machine_Learning\TM10007\brats\load_data.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(data2)


#### Split data in X and y
Split in X (data) and y (label)

In [4]:
# split column label from dataset X
y = X.pop('label')

#### Split data in train and test set
This function creates a panda dataframe and splits the data into test and train components.
This is done with test_size variable and the function train_test_split from the sklearn module.
Returns a train set with the data of 80% and a test set of 20% of the subjects.



In [5]:
# Split data in train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)  


#### Preprocessing : 

In [6]:
# infinity to NaN
X_train[X_train==np.inf]=np.nan
X_test[X_test==np.inf]=np.nan

# non-numeric features to NaN
X_train = X_train.replace(['#DIV/0!'], np.nan)
X_train = X_train.apply(pd.to_numeric, errors='coerce')

X_test  = X_test.replace(['#DIV/0!'], np.nan)
X_test = X_test.apply(pd.to_numeric, errors='coerce')

# If the total number of NaN observations in a column are greater than 40%, delete the entire column.
perc = 40.0
min_count = int(((100-perc)/100)*X_train.shape[0] + 1)
X_train_drop = X_train.dropna(axis=1, thresh=min_count)
X_labels = X_train_drop.keys()

X_test_drop = X_test[X_labels]

# fill the NaN observations.
data_fill_train = X_train_drop.fillna(X_train_drop.mean()) 
data_fill_test = X_test_drop.fillna(X_test_drop.mean()) 

#### Scale features

In [7]:
# robustscaler
scaler = RobustScaler()
scaler.fit(data_fill_train)
X_train_scaled = scaler.transform(data_fill_train)
X_test_scaled = scaler.transform(data_fill_test)

#### Transform features

In [8]:
# Perform a PCA
pca = decomposition.PCA(n_components=5)
pca.fit(X_train_scaled) 
X_train_pca = pca.transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)


#### Test Classifiers

In [ ]:
#Confusion matrix, Accuracy, sensitivity and specificity
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
classifiers = (KNeighborsClassifier(), RandomForestClassifier(), SVC(kernel="rbf", C=0.025, probability=True), NuSVC(probability=True))

for cls in classifiers:
    model = cls
    model.fit(X_train_pca, y_train)

    preds = model.predict(X_train_pca)

    metrics.plot_confusion_matrix(model, X_train_pca, y_train)
    display_labels =['Negative', 'Positive']


    confusion = metrics.confusion_matrix(y_train, preds)
    accuracy = metrics.accuracy_score(y_train, preds)
    sensitivity = metrics.recall_score(y_train, preds, pos_label='GBM')
    specificity = metrics.recall_score(y_train, preds, pos_label='LGG')

    #table = {'classifier': cls, 'accuracy': accuracy, 'Sensitivity', sensitivity, 'Specificity': specificity}
    print(f'For classifier: {cls}, Accuracy: {accuracy}, Sensitivity: {sensitivity}, Specificity{specificity}')



#### Cross validation

In [ ]:
# Create a cross-validation object

cv = KFold(n_splits=5, shuffle= True, random_state = 1)

X = X_train_pca

y = y_train.values
y = np.where(y=='GBM', 1, y)
y = np.where(y=='LGG', 2, y)
y = y.tolist()

classifiers = (KNeighborsClassifier(), RandomForestClassifier(), SVC(kernel="rbf", C=0.025, probability=True), NuSVC(probability=True), DecisionTreeClassifier() )

score_dict = {}
for cls in classifiers:

    list_scores = cross_val_score(cls, X, y, scoring = 'accuracy', cv=cv)
    mean = np.mean(list_scores)
    #print(f'The accuracy of {cls} is: {list_scores}')
    print(f'The mean accuracy of classifier {cls}: {mean}')
    score_dict[cls] = mean

best_performing = max(score_dict, key=score_dict.get)
value_best_performing = score_dict[best_performing]
print(f'The best performing classifier is {best_performing} with mean value {value_best_performing}')

#### Random forest model with hyperparameters 
First define some hyperparameters 


In [9]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 80, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree 
#max_depth = [2, 4]
# Minimum number of samples required to split a node
#min_sample_split = [2, 5]
# Minimum number of samples required at each leaf node
#min_sample_leaf = [1, 2]
# Method of selecting samples for training each tree
#bootstrap = [True, False]

# Create a dictionary of the hyperparameters above (create a parameter grid)
param_grid = {
    'n_estimators': n_estimators, 
    'max_features': max_features} 

print(param_grid)

# param_grid = {
#     'n_estimators': n_estimators, 
#     'max_features': max_features, 
#     'max_depth':max_depth, 
#     'min_sample_split': min_sample_split, 
#     'min_sample_leaf':min_sample_leaf, 
#     'bootstrap': bootstrap }
# print(param_grid)

# Create an insance of the classifier
rf_model = RandomForestClassifier()

{'n_estimators': [10, 17, 25, 33, 41, 48, 56, 64, 72, 80], 'max_features': ['auto', 'sqrt'], 'max_depth': [2, 4], 'min_sample_split': [2, 5], 'min_sample_leaf': [1, 2], 'bootstrap': [True, False]}


Doing a Gridsearch 

In [ ]:
# We use the RF model, take the hyperparameters, cv = amount of folds, 
#rf_grid = GridSearchCV(estimator = rf_model, param_grid = param_grid, cv = 10, verbose = 2, n_jobs = 4)
#rf_grid.fit(X_train_pca, y_train)
#rf_grid.best_params_




In [10]:
# Doing a random grid search 
# A randomised grid search will perform worse than a gridsearch but it is faster 
rf_random_grid = RandomizedSearchCV(estimator = rf_model, param_distributions = param_grid, cv = 10, verbose = 2, n_jobs = 4)
rf_random_grid.fit(X_train_pca, y_train)
rf_random_grid.best_params_

#print (f' Train Accuracy: {rf_grid.score(X_train_pca, y_train):.3f}')
#print (f' Test Accuracy: {rf_grid.score(X_test_pca, y_test):.3f}')

print (f' Test Accuracy: {rf_random_grid.score(X_train_pca, y_train):.3f}')
print (f' Test Accuracy: {rf_random_grid.score(X_test_pca, y_test):.3f}')



Fitting 10 folds for each of 10 candidates, totalling 100 fits
